In [ ]:
import os
import time
from openai import OpenAI
import json
import pandas as pd
import numpy as np
import faiss
import openpyxl

In [ ]:
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
client.models.list()

In [ ]:
def phase1(user_input):
    # instruction = """
    #     # Objective:
    #     As an intelligent customer service chatbot of the “Flora” focus app , my goal is to understand and help you solve problems encountered when using flora app effectively.
    #     # Main features of the Flora app:
    #     Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what’s more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track.* * Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most.
    #     # Support Process:
    #     Issue Identification: To ensure that I understand your problems, I will summarize the constructive, non-emotional parts of our input text in your tone into one key point. but if one key point is not enough to describe problem, key point can be more than one. Avoid using conjunctions. Then, I will and must only output these key points in json format.
    #     # Example:
    #     input: How do I change my password?
    #     output: {"keypoints": ["my problem is I need to change password."]}
    #     input: I am just doing my a-levels at the moment and I love using your app for revision. However, when I went onto the app earlier it logged me out and all my data had been lost. I was just wondering if you could help me out.
    #     output: {"keypoints": ["My data lost after being logged out of the app."]}
    #     input: My flora friend page won't load. If I delete the app and re download it and then login. Will all my data and garden time be lost?
    #     output: {"keypoints": ["my Flora friend page will not load.","I am concern about losing data and garden time after deleting and reinstalling the app."]}
    # """
    
    instruction = """
        # Objective:
        As an intelligent customer service chatbot of the “Flora” focus app , my goal is to understand and help you solve problems encountered when using flora app effectively.
        # Main features of the Flora app:
        Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what’s more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track.* * Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most.
        # Support Process:
        1.Issue Identification: To ensure I fully understand your problem, I will ask follow-up questions if your issue is unclear or if the reason behind your problem is not fully explained.
        2.Issue Summarization: After clearly understanding your problem, I will summarize the clear and constructive parts of your input in your tone into key points. If a single key point is insufficient to describe the problem, multiple key points will be used. I will avoid using conjunctions. Finally, I will only output these key points in JSON format.
        # Example:
        input: How do I change my password?
        output: {"keypoints": ["my problem is I need to change password."]}
        input: I am just doing my a-levels at the moment and I love using your app for revision. However, when I went onto the app earlier it logged me out and all my data had been lost. I was just wondering if you could help me out.
        output: {"keypoints": ["My data lost after being logged out of the app."]}
        input: My flora friend page won't load. If I delete the app and re download it and then login. Will all my data and garden time be lost?
        output: {"keypoints": ["my Flora friend page will not load.","I am concern about losing data and garden time after deleting and reinstalling the app."]}
        input: The app is full of bugs.
        output: Thanks for your feedback. Would you mind providing more details about the glitchy so we can provide better support?
    """
    
    # print("phase1 instruction: ")
    # print(instruction)

    assistant = client.beta.assistants.create(
        name = "Intelligent customer service chatbot",
        instructions = instruction,
        # model = "gpt-3.5-turbo-1106"
        model = "gpt-4o-mini"
    )

    thread = client.beta.threads.create()

    message = client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = user_input
    )

    run = client.beta.threads.runs.create(
        thread_id = thread.id,
        assistant_id = assistant.id
    )

    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id = thread.id,
            run_id = run.id
        )
    
    # need to handle run error

    messages = client.beta.threads.messages.list(
        thread_id = thread.id,
        order = 'asc'
    )

    conversation_log = {"Past Converstion Log": []}

    user_or_assistant = 0
    for i in messages.data[-10:]:
        role = "user" if user_or_assistant == 0 else "assistant"

        message = i.content[0].text.value.replace("\n", " ")
        
        conversation_log["Past Converstion Log"].append({
            "role": role,
            "message": message
        })
        
        user_or_assistant = 1 - user_or_assistant

    conversation_log_json = json.dumps(conversation_log, ensure_ascii=False, indent=4)

    keypoints_list = []

    last_message = conversation_log["Past Converstion Log"][-1]["message"]
    try: 
        keypoints = json.loads(last_message)["keypoints"]
        for keypoint in keypoints:
            keypoints_list.append(keypoint)
    except json.JSONDecodeError as e:
        print("No key point")

    # try:
    #     data = json.loads(conversation_log_json)
    #     for entry in data["Past Converstion Log"]:
    #         if entry["role"] == "assistant":
    #             message_data = json.loads(entry["message"])
    #             keypoints.extend(message_data.get("keypoints", []))
    # except json.JSONDecodeError as e:
    #     print("Unclear")

    return conversation_log_json, keypoints_list

# Phase 2

In [ ]:
def app_function_1():
    pass

def app_function_2(tour_name):
    pass

def app_function_3():
    pass

def app_function_4():
    pass

def app_function_5():
    pass

def app_function_6():
    pass

In [ ]:
csv_file = '../data/QK_embeddings0715.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

In [ ]:
def get_similar_QAK(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    key_point = np.array(response.data[0].embedding, ndmin=2)
    distances, indices = index.search(key_point, 3)
    QAK_df = pd.read_csv('../data/QAK0715.csv')
    QAK_array = QAK_df.to_numpy()
    QAKD = [(QAK_array[i], float(dist)) for dist, i in zip(distances[0], indices[0])]

    top3 = []
    for i in range(0,3):
        candidate = {}
        candidate["problem"] = QAKD[i][0][0].replace("\"", "\'")
        candidate["summarized keypoints"] = QAKD[i][0][2][16:-3].replace("\"", "\"")
        candidate["solution"] = {"form":QAKD[i][0][3], "content":QAKD[i][0][1].replace("\"", "\'")}
        top3.append(candidate)

    return top3

In [ ]:
def phase2(phase1_conversation_log, keypoint):
    instruction = """
    {
        "objective": "Act as an intelligent customer service chatbot for the 'Flora' focus app.",
        "job": [
            "1. Decide which candidate problem-resolution approach can be applied to solve the user's main problem. Note that the conversation log may contain irrelevant information and you should only use it as a clue for solving the user's main problem. First, if user's main problem is not related to Flora, you are prohibited to answer and must reply only with a special mark: '*'. Second, if the summarized key points within the three problem-resolution approaches are not related to the user's main problem, you are prohibited from answering and must reply only with a special mark: '*'. Third, if there are multiple related approaches, choose the one that is the most suitable.",
            "2. Based on the chosen problem-resolution approach, chat with user to solve the main problem without any format. If the approach contains an answer directly, follow the answer to interact with the user. You must not deviate from the chat based on the answer. If the approach contains a function, execute the exact function and continue to interact with the user.",
            "3. If the user asks another question different from the user's main problem either during or after the problem resolution process in Step 2, prompt the user to go back to the 'Feedback Portal' system to officially file the problem."
        ],
        "Main features of the Flora app": "Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what's more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track. Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most."
    }
    """

    # print("phase2 instruction: ")
    # print(instruction)

    tools = [
        {
            "type": "function",
            "function": {
                "name": "function_1",
                "description": "If the chosen problem-resolution approach contain the function: function_1, This function must be called. The return of function_1 is a workflow in the form of pseudo code and you must follow.",
            },
        },
        {
            "type": "function",
            "function": {
                "name": "external_1",
                "description": "This function must be called by function_1 function."
            },
        },

        {
            "type": "function",
            "function": {
                "name": "function_2",
                "description": "If the chosen problem-resolution approach contain the function: function_2, This function must be called. The return of function_2 is a workflow in the form of pseudo code and you must follow.",
            },
        },
        {
            "type": "function",
            "function": {
                "name": "external_2",
                "description": "This function must be called by function_2 function.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "tour_name": {
                            "type": "string",
                            "description": "Tour's name in Flora app. This must be gained from asking the user.",
                        },
                    },
                    "required": ["tour_name"],
                },
            },
        },

        {
            "type": "function",
            "function": {
                "name": "function_3",
                "description": "If the chosen problem-resolution approach contain the function: function_3, This function must be called. The return of function_3 is a workflow in the form of pseudo code and you must follow."
            },
        },
        {
            "type": "function",
            "function": {
                "name": "external_3",
                "description": "This function must be called by function_3 function."
            },
        },

        {
            "type": "function",
            "function": {
                "name": "function_4",
                "description": "If the chosen problem-resolution approach contain the function: function_4, This function must be called. The return of function_4 is a workflow in the form of pseudo code and you must follow."
            },
        },
        {
            "type": "function",
            "function": {
                "name": "external_4",
                "description": "This function must be called by function_4 function."
            },
        },

        {
            "type": "function",
            "function": {
                "name": "function_5",
                "description": "If the chosen problem-resolution approach contain the function: function_5, This function must be called. The return of function_5 is a workflow in the form of pseudo code and you must follow."
            },
        },
        {
            "type": "function",
            "function": {
                "name": "external_5",
                "description": "This function must be called by function_5 function."
            },
        },

        {
            "type": "function",
            "function": {
                "name": "function_6",
                "description": "If the chosen problem-resolution approach contain the function: function_6, This function must be called. The return of function_6 is a workflow in the form of pseudo code and you must follow."
            },
        },
        {
            "type": "function",
            "function": {
                "name": "external_6",
                "description": "This function must be called by function_6 function."
            },
        },
    ]

    assistant = client.beta.assistants.create(
        name = "customer service chatbot",
        instructions = instruction,
        # model = "gpt-3.5-turbo-1106",
        model = "gpt-4o-mini",
        tools = tools
    )

    thread = client.beta.threads.create()

    past_converation_log = json.loads(phase1_conversation_log)["Past Converstion Log"]
    past_converation_log[-1]["message"] = "This is my summarized key points: " + past_converation_log[-1]["message"] + ", and i will focus on main problem and help you solve it."

    user_prompt = {
        "Main Problem": keypoint,
        "Past Converstion Log": past_converation_log,
        "Candidate Problem-Resolution Approaches": get_similar_QAK(keypoint)
    }

    user_prompt_json = json.dumps(user_prompt, ensure_ascii=False)
    
    # print("phase2 user_prompt: (auto feed to assistant)")
    print(user_prompt_json)
    
    run_tools = []
    for candidate in user_prompt["Candidate Problem-Resolution Approaches"]:
        if candidate["solution"]["content"] == "function_1":
            function_1 = tools[0]
            if not (function_1 in run_tools):
                run_tools.append(function_1)

        if candidate["solution"]["content"] == "function_2":
            function_2 = tools[2]
            if not (function_2 in run_tools):
                run_tools.append(function_2)

        if candidate["solution"]["content"] == "function_3":
            function_3 = tools[4]
            if not (function_3 in run_tools):
                run_tools.append(function_3)

        if candidate["solution"]["content"] == "function_4":
            function_4 = tools[6]
            if not (function_4 in run_tools):
                run_tools.append(function_4)
        
        if candidate["solution"]["content"] == "function_5":
            function_5 = tools[8]
            if not (function_5 in run_tools):
                run_tools.append(function_5)
        
        if candidate["solution"]["content"] == "function_6":
            function_6 = tools[10]
            if not (function_6 in run_tools):
                run_tools.append(function_6)

    # print("run_tools: ")
    # for tool in run_tools:
    #     print(tool)

    message = client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = json.dumps(user_prompt, ensure_ascii=False)
    )

    run = client.beta.threads.runs.create(
        thread_id = thread.id,
        assistant_id = assistant.id,
        tools = run_tools,
        parallel_tool_calls = False
    )
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id = thread.id,
            run_id = run.id
        )
    # print(run.status)

    call_function = False
    if run.status == "requires_action":
        call_function = True
        for func in run.required_action.submit_tool_outputs.tool_calls:
            print(func)
        
        tool_output = []
        for func in run.required_action.submit_tool_outputs.tool_calls:
            if func.function.name == "function_1":
                flow = """
                    Tell user 'The Hawaii tour is a special gift for users who appreciate us! After planting a few trees, a window will appear, requesting your feedback. If you click "Like," you stand a chance to win it! If you haven't encountered this window in a while, you might want to consider reinstalling the app to see if it pops up again.';
                    Ask user if the problem has been resolved;
                    if(the problem has NOT been resolved) {call external_1 function;} 
                    else{Response user 'Have a nice day!';}
                """
                tmp = {"tool_call_id": func.id, "output": flow}
                tool_output.append(tmp)
            if func.function.name == "external_1":
                app_function_1()
                tmp = {"tool_call_id": func.id, "output": "We have manually unlocked the tour for your account. Please logout and login again to see if that works for you. Have a nice day!"}
                tool_output.append(tmp)

            if func.function.name == "function_2":
                flow = """
                    Tell user 'Please provide the tour name that you want to recalculate its progress.';
                    if(get tour name) {Call external_2;}
                    else {Ask Tour Name again.}
                """
                tmp = {"tool_call_id": func.id, "output": flow}
                tool_output.append(tmp)
            if func.function.name == "external_2":
                tour_name = eval(func.function.arguments)["tour_name"]
                app_function_2(tour_name)
                tmp = {"tool_call_id": func.id, "output": "We have recalculated the tour progress for your account. Please logout and login again to see if that works for you. Have a nice day!"}
                tool_output.append(tmp)

            if func.function.name == "function_3":
                flow = """
                    Response user “Hi, if your tree is falsely killed, you can first try to report a false detection by one of the following method:
                        1) press the '!' (triangular exclamation) button at the upper-right corner of the red screen after the focus session failed. 
                        2) Click on the "more" (three dots) button on the top-right corner of the failed story, and select "Report false detection".
                        If verifying that it is indeed a false detection, we will revive your tree and restore your focus time. Please note this function requires internet connection, and may take about a day for our system to process.
                        We highly recommend trying the above method first, and see if the issue can be resolved. However, If the issue persists or you think the method above does not fit your need, please let us know. Thanks for your understanding!”
                    if (the problem has NOT been resolved) {call external_3 function;}
                    else {Response user 'Have a nice day!';}
                """
                tmp = {"tool_call_id": func.id, "output": flow}
                tool_output.append(tmp)
            if func.function.name == "external_3":
                app_function_3()
                tmp = {"tool_call_id": func.id, "output": "We have tried to restore your falsely killed trees, add the missing stories and focus time to your account. Please logout and login again and see if the results are correct. Let us know if you have any questions. Have a nice day!"}
                tool_output.append(tmp)

            if func.function.name == "function_4":
                flow = """
                    Response user "Do you want to recalculate the total focus time of your account?";
                    if(user is consent) {call external_4 function;}
                    else {Response user "Please go back to the feedback portal and clarify your issue.";}
                """
                tmp = {"tool_call_id": func.id, "output": flow}
                tool_output.append(tmp)
            if func.function.name == "external_4":
                app_function_4()
                tmp = {"tool_call_id": func.id, "output": "We have recalculated the total focus time for your account. Please logout and login again to see if that works for you. Have a nice day!"}
                tool_output.append(tmp)

            if func.function.name == "function_5":
                flow = """
                    Response user "Are you sure you want to reset the progress of all tours? Since the progress is cached on your device, you may need to reinstall Flora to see the updated progress.";
                    if (user is consent) {call external_5 function;}
                    else {Response user "Please go back to the feedback portal and clarify your issue.";}
                """
                tmp = {"tool_call_id": func.id, "output": flow}
                tool_output.append(tmp)
            if func.function.name == "external_5":
                app_function_5()
                tmp = {"tool_call_id": func.id, "output": "Your progress record has been reset. Please reinstall Flora to see the updated progress."}
                tool_output.append(tmp)

            if func.function.name == "function_6":
                flow = """
                    Response user "Currently, we can only cancel all pending friend requests of an account. Are you sure you want to cancel all your pending friend requests?";
                    if (user is consent) {call external_6 function;}
                    else {Response user "Please go back to the feedback portal and clarify your issue.";}
                """
                tmp = {"tool_call_id": func.id, "output": flow}
                tool_output.append(tmp)
            if func.function.name == "external_6":
                app_function_6()
                tmp = {"tool_call_id": func.id, "output": "Your friend requests have been canceled."}
                tool_output.append(tmp)

        # print("tool_output:")
        # print(tool_output)
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id = thread.id,
            run_id = run.id,
            tool_outputs = tool_output
        )
        while run.status == "queued" or run.status == "in_progress":
            run = client.beta.threads.runs.retrieve(
                thread_id = thread.id,
                run_id = run.id
            )
        
    messages = client.beta.threads.messages.list(
        thread_id = thread.id,
        order = 'asc'
    )

    user_or_assistant = 0
    conversation_log = []

    for i in messages.data:
        role = "user" if user_or_assistant == 0 else "assistant"
        message = i.content[0].text.value.replace("\n", " ")
        conversation_log.append({"role": role, "message": message})
        user_or_assistant = 1 - user_or_assistant
    
    return conversation_log, call_function
        

# Test

In [ ]:
phase1_conversation_log, keypoints = phase1("should allow anyone to delete accidental trees")
print(json.loads(phase1_conversation_log)["Past Converstion Log"][-1]["message"])
print(keypoints)

In [ ]:
for keypoint in keypoints:
    print("key point: " + keypoint)
    phase2_conversation_log, call_function = phase2(phase1_conversation_log, keypoint)
    print("call_function: " + str(call_function))
    print("phase2_conversation_log: ")
    print(phase2_conversation_log[0])
    print(phase2_conversation_log[1])

# Experiment

In [ ]:
workbook = openpyxl.load_workbook('../data/ExampleFunctionCall.xlsx')
sheet = workbook['Testset Result']

In [ ]:
for i in range(2, 210):
    data = []
    data.append(sheet['A'+str(i)].value)
    data.append(sheet['B'+str(i)].value)
    data.append(sheet['C'+str(i)].value)

    phase1_conversation_log, keypoints = phase1(sheet['A'+str(i)].value)
    if len(keypoints) != 0:
        for keypoint in keypoints:
            phase2_conversation_log, call_function = phase2(phase1_conversation_log, keypoint)
            data.append(call_function)
            data.append(keypoint)
            data.append(phase2_conversation_log[-1]["message"])
    else:
        data.append(False)
        data.append(json.loads(phase1_conversation_log)["Past Converstion Log"][-1]["message"])

    for col_num, value in enumerate(data, start=1):
        sheet.cell(row=i, column=col_num, value=str(value))

    if i%10 == 0:
        workbook.save('ExampleFunctionCall.xlsx')
        
workbook.save('ExampleFunctionCall.xlsx')

In [139]:
assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)

for assistant in assistants.data:
    client.beta.assistants.delete(assistant.id)